In [2]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

import pinecone
loader = PyPDFLoader("datascience.pdf")


c:\Users\91849\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
data = loader.load()


In [7]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[30].page_content)} characters in your document')

You have 126 document(s) in your data
There are 2812 characters in your document


In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)


print (f'Now you have {len(texts)} documents')


In [14]:
import os
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

# Load environment variables from the .env file
with open('.env', 'r') as env_file:
    for line in env_file:
        variable, value = line.strip().split('=')
        os.environ[variable] = value

# Access environment variables as needed
Openai_api_key = os.environ.get('openai_api_key', '')
pinecone_api_key = os.environ.get('PINECONE_API_KEY', '')
pinecone_environment = os.environ.get('PINECONE_ENVIRONMENT', '')

# Initialize Pinecone with the loaded API key and environment
pinecone.init(api_key=pinecone_api_key, environment=pinecone_environment)

# Use the loaded OpenAI API key in your OpenAIEmbeddings constructor
embeddings = OpenAIEmbeddings(openai_api_key=Openai_api_key)


In [9]:
index_name="pine"
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)
query = "How does data science actually works?"
docs = docsearch.similarity_search(query)

In [ ]:
print(docs[0].page_content[:1000])


How does Data Science  
Actually Work?
It’s not rocket science… it’s something better - Data Science
Let’s not kid ourselves - Data Science is a complex field. It is difficult, 
intellectually taxing work, which requires the sophisticated integration 
of talent, tools and techniques. But as a field guide, we need to cut 
through the complexity and provide a clear, yet effective way to 
understand this new world. 
To do this, we will transform the field of Data Science into a set of 
simplified activities as shown in the figure, The Four Key Activities of a 
Data Science Endeavor . Data Science purists will likely disagree with 
this approach, but then again, they probably don’t need a field guide, 
sitting as they do in their ivory towers! In the real world, we need 
clear and simple operating models to help drive us forward. 
THE FIELD  GUIDE to  DATA  SCIENCE


In [10]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain


In [30]:
import os 
with open('.env', 'r') as env_file:
    for line in env_file:
        variable, value = line.strip().split('=')
        os.environ[variable] = value

# Access environment variables as needed
Openai_api_key = os.environ.get('openai_api_key', '')
llm = OpenAI(temperature=0, openai_api_key=Openai_api_key)
chain = load_qa_chain(llm, chain_type="stuff")

docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

query = "is data science a threat ?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)


' No, data science is not a threat. It is a tool that can be used to create insights and help us better understand the world around us.'